In [3]:
!echo Installing bbSearch module from web ...
!echo creating bbmodcache subfolder
!mkdir -p bbmodcache
!echo downloading bbSearch module
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py

from bbmodcache.bbSearch import SearchProblem, search

Installing bbSearch module from web ...
creating bbmodcache subfolder
downloading bbSearch module
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18767  100 18767    0     0   374k      0 --:--:-- --:--:-- --:--:--  374k
Loading bbSearch Version 2.1 (at 17:13, Thu 20 Feb)
Last module source code edit 9am Thursday 24th Feb 2022


In [4]:
def parcel_heuristic(parcel):
    """Compute a score for each parcel to prioritize selection."""
    is_first_class = 1 if parcel.priority == 1 else 0
    is_stored_long = 1 if parcel.storage_days() >= 3 else 0
    return (10 * is_first_class) + (5 * is_stored_long)

In [ ]:
from copy import deepcopy
from datetime import datetime, timedelta
import random


def generate_random_date():
    days_ago = random.randint(1, 7)
    return datetime.now() - timedelta(days=days_ago)
def least_full_belt(conveyor_belts):
    return min(conveyor_belts, key=lambda cb: len(cb.parcels))


class Parcel:
    def __init__(self, parcel_id, priority):
        self.parcel_id = parcel_id
        self.priority = priority # 1 = first class, 2 = second class

    def __repr__(self):
        return f"({self.parcel_id}, {self.priority})"
    
    def storage_days(self):
        return (datetime.now() - self.date).days

class Robot:
    def __init__(self, location, carried_parcels_robot, carried_parcels=None):
        if carried_parcels is None:
            carried_parcels = []
        self.location = location
        self.carried_parcels = carried_parcels
        self.carried_parcels_robot = carried_parcels_robot

    def __repr__(self):
        return f"Robot at {self.location}, Carrying: {self.carried_parcels}"

class State:
    def __init__(self, robot, storage_room):
        self.robot = robot
        self.storage_room = storage_room


    def __repr__(self):
        return f"Robot: {self.robot}, Storage: {self.storage_room}, Belts: {self.conveyor_belts}, Carried: {self.carried_parcels}"
 
STORAGE_ROOM = [
    Parcel(str(i), random.choice([1, 2]))  # Ensure ID is a string
    for i in range(101, 133)
]

class RobotWorker(SearchProblem):
    def __init__(self, state, remaining_parcels):
        self.initial_state = state
        self.remaining_parcels = remaining_parcels

    def possible_actions(self, state):
        actions = []

        if state.robot.location == 'storage room' and (state.robot.carried_parcels != None):
            for i in range (0, 5):
                actions.append(("pick up", state.storage_room[i]))

        return actions
    
    def successor(self, state, action):
        next_state = deepcopy(state)
        print(next_state.robot.carried_parcels)
        act, target = action
        target_id = target.parcel_id
        
        if state.robot.carried_parcels_robot is not []:
            next_state.robot.carried_parcels_robot = state.robot.carried_parcels_robot

        print(next_state.storage_room)

        if act == "pick up":
            print(target)
            for i in state.storage_room:
                if i.parcel_id == target_id:
                    state.storage_room.remove(i)
            state.robot.carried_parcels.append(target)

rob = Robot('storage room', None)
state = State(rob, STORAGE_ROOM)
RW_PROBLEM = RobotWorker(state, 0)

poss_acts = RW_PROBLEM.possible_actions(RW_PROBLEM.initial_state)
poss_acts

for acts in poss_acts:
    next_state = RW_PROBLEM.successor(RW_PROBLEM.initial_state, acts)
    RW_PROBLEM.display_state(next_state)
    print()

[]
[(101, 1), (102, 2), (103, 2), (104, 1), (105, 1), (106, 2), (107, 1), (108, 1), (109, 1), (110, 2), (111, 1), (112, 1), (113, 2), (114, 2), (115, 2), (116, 1), (117, 2), (118, 1), (119, 2), (120, 1), (121, 1), (122, 1), (123, 2), (124, 2), (125, 1), (126, 1), (127, 1), (128, 1), (129, 1), (130, 2), (131, 2), (132, 2)]
(101, 1)
None

[(101, 1)]
[(102, 2), (103, 2), (104, 1), (105, 1), (106, 2), (107, 1), (108, 1), (109, 1), (110, 2), (111, 1), (112, 1), (113, 2), (114, 2), (115, 2), (116, 1), (117, 2), (118, 1), (119, 2), (120, 1), (121, 1), (122, 1), (123, 2), (124, 2), (125, 1), (126, 1), (127, 1), (128, 1), (129, 1), (130, 2), (131, 2), (132, 2)]
(102, 2)
None

[(101, 1), (102, 2)]
[(103, 2), (104, 1), (105, 1), (106, 2), (107, 1), (108, 1), (109, 1), (110, 2), (111, 1), (112, 1), (113, 2), (114, 2), (115, 2), (116, 1), (117, 2), (118, 1), (119, 2), (120, 1), (121, 1), (122, 1), (123, 2), (124, 2), (125, 1), (126, 1), (127, 1), (128, 1), (129, 1), (130, 2), (131, 2), (132, 2)]
(1

In [ ]:
    def possible_actions(self, state):
        actions = []

        # pick up parcels from storage using heuristic
        if state.robot.location == 'storage room' and (len(state.robot.carried_parcels) == 0):
            sorted_parcels = sorted(state.storage_room, key=parcel_heuristic, reverse=True)
            for i in range(0, len(sorted_parcels)-1):
                if i < 5:
                    actions.append(("pick up", sorted_parcels[i]))

        # drop off parcels at conveyor belts
        for parcel in state.robot.carried_parcels:
            least_full = least_full_belt(state.conveyor_belts)
            if least_full.capacity > 0:
                print(f"\nMoving parcel {parcel.parcel_id} (Priority: {parcel.priority}, Date: {parcel.date}) to {least_full.name}")
                state.storage_room.remove(parcel)  # Remove parcel from storage
                least_full.add_parcel(parcel)  # Add parcel to the conveyor
            else:
                print(f"!!!All conveyors are full!!! Returning parcel {parcel.parcel_id} to storage.")
                state.storage_room.append(parcel)  # Return parcel to storage if all belts are full

        # move between locations
        for cb in state.conveyor_belts:
            actions.append(("move to", cb.name))
        actions.append(("move to", 'storage room'))

        return actions 


def successor(self, state, action):
        next_state = deepcopy(state)
        act, target = action
        
        if act == "pick up":
            next_state.robot.carried_parcels.append(target)
            # next_state.storage_room.remove(target)
            if target.parcel_id in [p.parcel_id for p in state.storage_room]:
                state.storage_room.remove(target)
            else:
                print(f"Parcel {target.parcel_id} not in storage.")
            print(f"\nRobot picks up parcel {target.parcel_id} from storage.")

        elif act == "drop off":
            next_state.robot.carried_parcels.remove(target)
            best_belt = least_full_belt(next_state.conveyor_belts)
            if best_belt.add_parcel(target):
                print(f"\nRobot drops off parcel {target.parcel_id} at {best_belt.name}.")
            else:
                print(f"\nAll conveyors full! Returning {target.parcel_id} to storage.")
                next_state.storage_room.add(target)

        elif act == "move to":
            next_state.robot.location = target
            print(f"\nRobot moves to {target}.")

        return next_state





search(RW_PROBLEM, 'BF/FIFO', 10000, loop_check=True)

# sorted via heuristic
sorted_parcels = sorted(STORAGE_ROOM, key=parcel_heuristic, reverse=True)
print("Initial Parcel Order:", [p.parcel_id for p in sorted_parcels])

# assign parcels to the conveyor belts
for parcel in sorted_parcels:
    # find most available capacity
    least_full = least_full_belt(CONVEYOR_BELTS)
    print(f"-" * 50)

    # check if current conveyor has capacity
    if least_full.capacity > 0:
        print(f"\nMoving parcel {parcel.parcel_id} (Priority: {parcel.priority}, Date: {parcel.date}) to {least_full.name}")
        STORAGE_ROOM.remove(parcel)  # Remove parcel from storage
        least_full.add_parcel(parcel)  # Add parcel to the conveyor
    else:
        print(f"!!!All conveyors are full!!! Returning parcel {parcel.parcel_id} to storage.")
        STORAGE_ROOM.append(parcel)  # Return parcel to storage if all belts are full


    #current state of conveyor belts
    for belt in CONVEYOR_BELTS:
        print(f"{belt.name} contents: {[f'({p.parcel_id}, {p.date}, {p.priority})' for p in belt.parcels]} Capacity: {belt.capacity}%")

    # remaining parcels in storage
    print("Remaining in Storage:", [p.parcel_id for p in STORAGE_ROOM],"\n")

# output final state
print("\n\nFinal Parcel States:")
for belt in CONVEYOR_BELTS:
    print(f"{belt.name}: {[p.parcel_id for p in belt.parcels]}")
print("Remaining in Storage:", [p.parcel_id for p in STORAGE_ROOM],"\n")